In [1]:
import sys

# variables
charClass = 0; lexeme = []; nextChar = ' '; nextToken = ' '; tokenString = ' '
index = 0; chars = []; lastIndex = 0; num = 0; arIndex = 0
funcName = []; localvarName = []; varName = []; localLine = []; AR = dict()
entireVal = []; syntax = True; error = False

# Character classes
LETTER = 0; DIGIT = 1; UNKNOWN = 2; EOF = 99

# Token codes
IDENT = 10; 
LEFT_BRACKET = 20; RIGHT_BRACKET = 21; SEMI_COLON = 22; COMMA = 23; 
VARIABLE = 30; CALL = 31; PRINT_ARI = 32;

In [2]:
def getChar():
    # 다음 char를 nextChar에 저장하고 charClass 결정
    global nextChar, charClass, index

    if index == lastIndex + 1:
        nextChar = "EOF"
    else:
        nextChar = chars[index]
    index = index + 1
    
    if nextChar == "EOF":
        charClass = EOF
    elif nextChar != ' ' and nextChar != "\n":
        if nextChar.isalpha() or nextChar == '_':
            charClass = LETTER
        elif nextChar.isdigit():
            charClass = DIGIT
        else:
            charClass = UNKNOWN
    else:
        charClass = EOF

In [3]:
def addChar():
    # lexeme 배열에 nextChar를 덧붙임
    global lexeme
    lexeme.append(nextChar)

In [4]:
def lexical():
    # 식별자, 숫자, 연산자를 구별해서 nextToken에 셋 중에 어떤 것인지 저장
    global lexeme, nextToken, tokenString
    del lexeme[:]
    getNonBlank()
    
    # 처음 char가 문자로 시작하면 identifier를 의미함, lexeme 문자열을 tokenString에 저장
    if charClass == LETTER:
        addChar(); getChar()
        while charClass == LETTER or charClass == DIGIT:
            addChar(); getChar()
        tokenString = ''.join(lexeme)
        if tokenString == 'variable':
            nextToken = VARIABLE
        elif tokenString == 'call':
            nextToken = CALL
        elif tokenString == 'print_ari':
            nextToken = PRINT_ARI
        else:
            nextToken = IDENT
    
    # 처음 char가 문자, 숫자가 아닌 special character로 시작하면 op를 의미함, lookup 함수 실행
    elif charClass == UNKNOWN:
        lookup(); getChar()
    
    # end of file
    elif charClass == EOF:
        nextToken = EOF
        lexeme.append('EOF0')

In [5]:
def lookup():
    # 문자와 숫자가 아닌 char를 받았을 때 operator(연산자) 구분
    global nextToken
    if nextChar == '{':
        addChar(); nextToken = LEFT_BRACKET
    elif nextChar == '}':
        addChar(); nextToken = RIGHT_BRACKET
    elif nextChar == ',':
        addChar(); nextToken = COMMA
    elif nextChar == ';':
        addChar(); nextToken = SEMI_COLON
    else:
        addChar(); nextToken = EOF

In [6]:
def getNonBlank():
    # white-space 무시
    while nextChar.isspace():
        getChar()

In [7]:
def start():
    # <start> → <functions>
    functions()
    
def functions():
    # <functions> → <function> { <function> }
    function()
    while nextToken != EOF and syntax == True:
        function()
        
def function():
    # <function> → <identifier> { <function_body> }
    global num, localvarName, localLine, syntax
    localLine.append(num); localLine[num] = list()
    if nextToken == IDENT:
        lexical()
        if tokenString in funcName:
            syntax = False
            print('Duplicate declaration of the function name:', tokenString)
        elif tokenString in entireVal:
            syntax = False
            print('Duplicate declaration of the identifier or the function name:', tokenString)
        else:
            funcName.append(tokenString)
            if nextToken == LEFT_BRACKET:
                lexical()
                function_body()
                if nextToken == RIGHT_BRACKET:
                    lexical()
                else:
                    syntax = False
                    print('} 가 없다')
            else:
                syntax = False
                print('{ 가 없다')
                lexical()
    varName.append(num)
    varName[num] = localvarName
    num += 1
    localvarName = []
    
def function_body():
    if nextToken == VARIABLE:
        var_definitions()
    if syntax == True:
        statements();
    
def var_definitions():
    while nextToken == VARIABLE and syntax == True:
        lexical()
        var_definition()

def var_definition():
    global syntax, localvarName
    if tokenString in funcName:
        syntax = False
        print('Duplicate declaration of the identifier or the function name:', tokenString)
    else:
        if tokenString in localvarName:
            print('Duplicate declaration of the identifier:', tokenString)
        else:
            localvarName.append(tokenString)
            entireVal.append(tokenString)
        lexical()
        while nextToken == COMMA:
            lexical()
            if tokenString in funcName:
                syntax = False
                print('Duplicate declaration of the identifier or the function name:', tokenString)
            else:
                if tokenString in localvarName:
                    print('Duplicate declaration of the identifier:', tokenString)
                else:
                    localvarName.append(tokenString)
                    entireVal.append(tokenString)
                lexical()
        if nextToken == SEMI_COLON:
            lexical()
        else:
            syntax = False
            print('; 없다')
    
def statements():
    while nextToken != RIGHT_BRACKET and nextToken != EOF:
        statement()
    
def statement():
    # <statement> → call <identifier>; |  print_ari; | <identifier>; 
    global syntax
    oneLine = tokenString
    if nextToken == CALL:
        lexical()
        oneLine = oneLine + ' ' + tokenString
        lexical()
        if nextToken == SEMI_COLON:
            lexical()
        else:
            syntax = False
            print('; 없다')
    elif nextToken == PRINT_ARI:
        lexical()
        if nextToken == SEMI_COLON:
            lexical()
        else:
            syntax = False
            print('; 없다')
    elif nextToken == IDENT:
        lexical()
        if nextToken == SEMI_COLON:
            lexical()
        else:
            syntax = False
            print('; 없다')
    localLine[num].append(oneLine)
        

In [8]:
def run():
    global AR
    ARI = list()
    
    main = funcName.index('main')
    
    # AR에 main 함수의 변수 저장
    for i in varName[main]:
        ARI.append('Local variable: ' + str(i))
    AR['main'] = ARI
    
    for i in localLine[main]:
        if i.split()[0] == 'call':
            call('main', i.split()[1], localLine[main].index(i) + 1, arIndex)
        elif i == 'print_ari':
            printARI()
        else:
            varReference(main, i)

        if error == True:
            break
            # link count와 local offset 값 출력
            # link count는 링크 따라가는 횟수
            # local offset은 각 AR에서 몇 번째인지

In [9]:
def call(caller, called, returnAddress, dynamicLink):
    global AR, arIndex, error
    
    ARI = list()
    
    if called not in funcName:
        print('Call to undefined function:', called)
        error = True
    
    else:
        funcNum = funcName.index(called)
        # return address 저장 -> 원래 함수 호출문 바로 다음
        ARI.append('Return Address: ' + caller + ':' + str(returnAddress))
        # dynamic link 저장 -> caller의 AR 제일 하단 부분
        ARI.append('Dynamic Link: ' +  str(dynamicLink))
        # local variables 저장
        for i in varName[funcNum]:
            ARI.append('Local variable: ' + str(i))
        
        AR[called] = ARI
        
        if caller != 'main':
            arIndex += 2    
        arIndex += len(varName[funcName.index(caller)])
        
        for i in localLine[funcNum]:
            if i.split()[0] == 'call':
                call(called, i.split()[1], localLine[funcNum].index(i) + 1, arIndex)
            elif i == 'print_ari':
                printARI()
            else:
                varReference(funcNum, i)
            
            if error == True:
                break

In [10]:
def printARI():
    keys = list(); values = []
    for key in AR.items():
        keys.append(key)
    
    for key, value in keys[::-1]:
        print(key, end = ':')
        values = AR[key][::-1]
        for value in values:
            print('\t', value)
    
    print()

In [11]:
def varReference(funcNum, var):
    keys = list()
    # funcNum는 함수의 정의된 순서 번호, first, second, main 순서 -> second이므로 1
    # varName은 funcName과 인덱스가 같음
    # var은 변수명
    linkCount = 0
    funcNow = funcName[funcNum]
    
    for key, value in AR.items():
        keys.append(key)
    # keys에는 AR에 담겨있는 순서대로 함수명이 있다
    # 처음 main, first, second
    arNum = keys.index(funcNow) # -> second는 keys에서 3번째이므로 arNum은 2
    
    while True:
        if var in varName[funcNum]: # varName[1], 즉 second의 변수 리스트에 var이 있느냐
            break
        else:
            arNum -= 1
            funcNum = funcName.index(keys[arNum])
            # arNum에서 -1을 해서 AR 순서에서 하나 내려가서 first가 됨 즉, first의 정의된 인덱스를 funcNum에 저장
            linkCount += 1
            
    offset = varName[funcNum].index(var)
    
    if arNum != 0:
        offset += 2
    
    print(funcNow + ':', var, '=>', str(linkCount)+',', offset, '\n')

In [12]:
try:
#     with open(sys.argv[1], "r") as file:
    with open('txt1.txt', "r") as file:
        list_line = file.read()
        for line in range(0, len(list_line)):
            chars.append(list_line[line])
    
    lastIndex = len(list_line) - 1

    getChar()
    lexical()
    start()
    
    if syntax == False:
        print('Syntax Error')
    else:
        print('Syntax O.K.\n')
        
        if 'main' not in funcName:
            error = True
            print('No Starting function')
        
        if error == False:
            run()
    
        
except Exception as e:
    print('ERROR - ', e)
    sys.exit()

Syntax O.K.

second: b => 1, 3 

second:	 Local variable: a
	 Local variable: x
	 Dynamic Link: 2
	 Return Address: first:1
first:	 Local variable: c
	 Local variable: b
	 Local variable: a
	 Dynamic Link: 0
	 Return Address: main:1
main:	 Local variable: q
	 Local variable: p

second: q => 2, 1 

first: p => 1, 0 

